## Problem Statement

##### During lab 3.01 I observed a potential correlation between squarefeet and price. This to me is a natural occurrence because property represented as area represents the non-depreciative asset of land ownership. Addtionally, having many conversations with Bede, there became a focus on the relationships between quality/conditon and salesprice.

##### This raises the question: Are descriptors of depreciation (quality/cond) better estimators of sales price than descriptors of value (property size by sqft)? 

#### H0: The true mean difference in RMSE between models who have more Qual/Cond features than those who do not is 0. 

#### H1: The true mean difference in RMSE between models who have more Qual/Cond features than those who do not is NOT 0. 

> $H_0: \mu_\text{trl_RMSE} - \mu_\text{ctrl_RMSE} = 0$ <br>
> $H_A: \mu_\text{trt_RMSE} - \mu_\text{ctrl_RMSE}\ne0$ <br>
### $$P(\text{data}\;|\;H_0 \text{ true})$$
$$\alpha=0.05$$


Additional Experiment Information:
- To get the mean RMSE score for the two different kinds of models, we will run each model structure 3 times with the same 3 random_states on the test train split
- List of Random States used: [777, 42, 1]
- After each run, record different measurements and then calculate if the difference of mean RMSE beats the significant factor or not.

In [1]:
# operating system
import os
# package settings
from sklearn import set_config
# Data Manip & View
import pandas as pd
# Data Viz
import seaborn as sns
import matplotlib.pyplot as plt
# modeling process
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

# additional
import warnings

In [2]:
# set transformers to output as pandas dfs
set_config(transform_output="pandas")

### Now that we have a better understanding of the data we are working with let's do some preprocessing and feature engineering before making our models

In [2]:
# read in datasets
df_train = pd.read_csv('../datasets/cleaned_datasets/cleaned_train.csv', index_col=0)
df_test = pd.read_csv('../datasets/cleaned_datasets/cleaned_test.csv', index_col=0)

In [13]:
# set original dataframe index for future use
train_id = df_train['id']
test_id = df_test['id']

In [14]:
# examine shape
df_train.shape, df_test.shape

((2051, 77), (878, 76))

In [15]:
# create separate dataframes for features we are interested in hypothesis wise
# however as we saw in EDA, we'll probably need to pull in some of these features
# to help boost our model's performance

df_train_hyp = df_train.drop(columns = ['pid','ms_subclass','year_built','year_remod/add','bsmt_full_bath',
                                        'bsmt_half_bath','full_bath','half_bath','bedroom_abvgr','kitchen_abvgr',
                                        'totrms_abvgrd','fireplaces','garage_yr_blt','garage_cars','enclosed_porch',
                                        '3ssn_porch','screen_porch','misc_val','mo_sold','yr_sold','ms_zoning',
                                        'street','lot_shape','land_contour','utilities','lot_config','land_slope',
                                        'neighborhood','condition_1','condition_2','bldg_type','house_style',
                                        'roof_style','roof_matl','exterior_1st','exterior_2nd','mas_vnr_type',
                                        'foundation','bsmt_exposure','bsmtfin_type_1','bsmtfin_type_2','heating',
                                        'central_air','electrical','functional','garage_type','garage_finish',
                                        'paved_drive','sale_type'])

df_test_hyp = df_test.drop(columns = ['pid','ms_subclass','year_built','year_remod/add','bsmt_full_bath',
                                        'bsmt_half_bath','full_bath','half_bath','bedroom_abvgr','kitchen_abvgr',
                                        'totrms_abvgrd','fireplaces','garage_yr_blt','garage_cars','enclosed_porch',
                                        '3ssn_porch','screen_porch','misc_val','mo_sold','yr_sold','ms_zoning',
                                        'street','lot_shape','land_contour','utilities','lot_config','land_slope',
                                        'neighborhood','condition_1','condition_2','bldg_type','house_style',
                                        'roof_style','roof_matl','exterior_1st','exterior_2nd','mas_vnr_type',
                                        'foundation','bsmt_exposure','bsmtfin_type_1','bsmtfin_type_2','heating',
                                        'central_air','electrical','functional','garage_type','garage_finish',
                                        'paved_drive','sale_type'])

In [16]:
# Test shape of new hypothesis model dfs
df_train_hyp.shape, df_test_hyp.shape

((2051, 28), (878, 27))

In [38]:
# split dfs into their numerical and object forms for OHE
df_train_hyp_num = df_train_hyp.select_dtypes('number')
df_train_hyp_obj = df_train_hyp.select_dtypes('object')
df_test_hyp_num = df_test_hyp.select_dtypes('number')
df_test_hyp_obj = df_test_hyp.select_dtypes('object')

In [39]:
# Check for symmetry
print('this is train num', df_train_hyp_num.shape)
print('this is train ob', df_train_hyp_obj.shape)
print('this is test num', df_test_hyp_num.shape)
print('this is test ob', df_test_hyp_obj.shape)

this is train num (2051, 19)
this is train ob (2051, 9)
this is test num (878, 18)
this is test ob (878, 9)


In [29]:
# Instantiate OneHotEncoder for Categorical Features, dropping first row to apply inference to data set
ohe = OneHotEncoder(
    drop='first',
    sparse=False,
    handle_unknown='ignore'
)

# Instantiate StandardScaler
ss = StandardScaler()

# Instantiate PolynomialFeatures
poly = PolynomialFeatures(degree = 2,
                          interaction_only = True,
                          include_bias=False)

# Model 1:
- All Hypothesis Features despite multicolinearity
- OHE, SS, and Polynomial

## Preprocessing

In [41]:
# Fit Transform Train Set
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    df_train_hyp_obj = ohe.fit_transform(df_train_hyp_obj)

In [42]:
# Transform Test Set
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    df_test_hyp_obj = ohe.transform(df_test_hyp_obj)

In [43]:
# Add index back to object for merge using saved indexes
df_train_hyp_obj['id'] = train_id
df_test_hyp_obj['id'] = test_id

In [44]:
# Merge respective dfs back together for 
df_ohe_train = df_train_hyp_num.merge(df_train_hyp_obj, how='left', on='id')
df_ohe_test = df_test_hyp_num.merge(df_test_hyp_obj, how='left', on='id')

In [45]:
# Check Shapes after OHE
df_ohe_train.shape,df_ohe_test.shape

((2051, 54), (878, 53))

In [46]:
# set our X and Y
X = df_ohe_train.drop(columns=['saleprice'])
y = df_ohe_train['saleprice']

In [47]:
# Test Train Split, with different Random States to get mean RMSE for hypothesis
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=777)

In [48]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1538, 53), (513, 53), (1538,), (513,))

## Feature Engineering

### Poly - Increase number of Features by combining interactions only (ex. a*b, but not a**2)

In [49]:
X_train_poly = poly.fit_transform(X_train.drop(columns=['id']))

In [50]:
X_train_poly.shape

(1538, 1378)

In [51]:
X_test_poly = poly.transform(X_test.drop(columns=['id']))

In [52]:
X_test_poly.shape

(513, 1378)

### Standard Scaler - Transform all numerical data points to be on the same scale

In [53]:
X_train_ss = ss.fit_transform(X_train_poly)

In [54]:
X_train_ss.shape

(1538, 1378)

In [55]:
X_test_ss = ss.transform(X_test_poly)

In [56]:
X_test_ss.shape

(513, 1378)

### Baseline Model - Plot the mean of y to better understand our future model's perfomance

In [77]:
base_1 = [y_train.mean()]*len(y_test)

In [78]:
r2_score(y_test, base_1)

-0.00018676221445179664

#### Looking at our Baseline model for the first go around, we can't do much worse than it. As the baseline is only the mean of the y it is heavily underfit to the data and therefore negative.

### Modeling

Multi Linear Regression

In [57]:
# Instantiate Model
lr = LinearRegression()

In [58]:
# Fit Model
lr.fit(X_train_ss, y_train)

LinearRegression()

Calculate R2 scores

In [59]:
lr.score(X_train_ss, y_train)

0.9592892516217444

In [60]:
lr.score(X_test_ss, y_test)

-3.623684659803418e+29

Calculate RMSE on both Train and Test

In [61]:
mean_squared_error(y_train, lr.predict(X_train_ss), squared = False)

15946.81406768557

In [62]:
mean_squared_error(y_test, lr.predict(X_test_ss), squared = False)

4.806328309389104e+19

In [64]:
print(f'intercept: {lr.intercept_}')
print(f'coeficient: {lr.coef_}')

intercept: 181148.55005653374
coeficient: [ 2.13723530e+17  2.45550005e+15 -7.90341939e+15 ...  0.00000000e+00
  0.00000000e+00  0.00000000e+00]


Based on our Intercepts and coefficients, when x = 0 our base price is ~181,148.55

In [66]:
cross_val_score(lr, X_train, y_train, cv = 5).mean()

0.8116405776931652

#### Observations:
- ##### Our first model is very overfit to our training data with a lot of bias
- ##### There is also a lot of variance as well with our test score above the billions in RMSE
- ##### However when we cross val score our train data, our score isnt too bad? It suggests that with a cv split of 5, that we can generalise 85% over the whole dataset
- ##### Before we move on to creating our hypothesis models and testing our hypothesis, let's see if Ridge, Lasso, and ElasticNet can help this original model

### CV Models

In [80]:
alphas = np.logspace(-10, 5, 400)

### LinearRegression - RidgeCV

In [81]:
rg_cv = RidgeCV(alphas=alphas)

In [83]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    rg_cv.fit(X_train_ss, y_train);

In [84]:
# Best Alpha
rg_cv.alpha_

392.631340170684

In [85]:
print(f"""
training r2: {rg_cv.score(X_train_ss, y_train)}
validation r2: {rg_cv.score(X_test_ss, y_test)}
cross val score: {cross_val_score(RidgeCV(alphas = rg_cv.alpha_), X_train_ss, y_train).mean()}

training RMSE: {mean_squared_error(y_train, rg_cv.predict(X_train_ss), squared=False)}
validation RMSE: {mean_squared_error(y_test, rg_cv.predict(X_test_ss), squared=False)}
""")


training r2: 0.9335206571091633
validation r2: 0.8715290300505348
cross val score: 0.7923631732709664

training RMSE: 20378.055776621502
validation RMSE: 28618.095982381557



#### Observations:
- #### Ridge definitely made a big improvment on model performance over mlr; bringing our R2 scores on training and test much closer to each other but still exhibiting an overfit model with a lot of bias and variance. 
- #### Our RMSE is also in a much better place, while still experience a large amount of variance
- #### Will be interesting to see how much better Ridge performs when we are more selective with what features we include in regards to multicolinearity seen in our EDA.

### LinearRegression - LassoCV

In [86]:
l_alphas = np.arange(0, 1, .01)

In [87]:
ls_cv = LassoCV(alphas=l_alphas, n_jobs=-1)

In [88]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    ls_cv.fit(X_train_ss, y_train)

In [89]:
ls_cv.alpha_

0.8200000000000001

In [91]:
# with warnings.catch_warnings():
#     warnings.simplefilter('ignore')
    
#     ls_cv.fit(X_train_ss, y_train)
#     print(f"""
#     training r2: {ls_cv.score(X_train_ss, y_train)}
#     validation r2: {ls_cv.score(X_test_ss, y_test)}
#     cross val score: {cross_val_score(LassoCV(alphas = alphas, n_jobs=-1), X_train_ss, y_train).mean()}

#     training RMSE: {mean_squared_error(y_train, ls_cv.predict(X_train_ss), squared=False)}
#     validation RMSE: {mean_squared_error(y_test, ls_cv.predict(X_test_ss), squared=False)}
#     """)

The model is not well defined in that we cannot complete an actual LassoCV Model, let's see if ElasticNetCV can finish scoring.

### LinearRegression - ElasticNetCV

In [190]:
# Instantiate ElasticNetCV
enet_cv = ElasticNetCV(
    l1_ratio = [.04, .05, .06, .07],
    alphas = np.logspace(-10, 5, 200),
    cv = 5,
    n_jobs=-1
)

# Instantiate ElasticNetCV
enet_cv1 = ElasticNetCV(
    l1_ratio = [.04, .05, .06, .07],
    alphas = np.logspace(-10, 5, 200),
    cv = 5,
    n_jobs=-1
)

In [1]:
# with warnings.catch_warnings():
#     warnings.simplefilter('ignore')
    
#     enet_cv.fit(X_train_ss, y_train);

Similar to our LassoCV run, this model with all the features we are interested in is not well defined. Possible explanations as to why both LassoCv and ElasticNetCV failed was because of the multicollinearity present in our dataset affecting the gradient and causing a convergence to never appear.

## Hypothesis Testing Time!

In [9]:
df_train_hyp.columns

Index(['id', 'lot_frontage', 'lot_area', 'overall_qual', 'overall_cond',
       'mas_vnr_area', 'bsmtfin_sf_1', 'bsmtfin_sf_2', 'bsmt_unf_sf',
       'total_bsmt_sf', '1st_flr_sf', '2nd_flr_sf', 'low_qual_fin_sf',
       'gr_liv_area', 'garage_area', 'wood_deck_sf', 'open_porch_sf',
       'pool_area', 'saleprice', 'exter_qual', 'exter_cond', 'bsmt_qual',
       'bsmt_cond', 'heating_qc', 'kitchen_qual', 'fireplace_qu',
       'garage_qual', 'garage_cond'],
      dtype='object')

Using our EDA and Visualizations, let's create better defined models for our hypothesis testing:

---
For our Model with More Quality/Condition Features than SF Features, will contain:<br>
Quality/Condition: `overall_qual`, `overall_cond`, `kitchen_qual`, `heating_qc`, `fireplace_qu`, `bsmt_qual`<br>
Squarefeet: `wood_deck_sf`,`lot_area`,`lot_frontage`,`mas_vnr_area`<br>

These features were selected by examining the EDA as well as trying to reduce the multicollinearity we had in the original model. Each numerical feature that was selected above had both a positive correlation with price and a VIF score of <5. Hoping this allows us to finish both LassoCV and ElasticNetCV<br>

10 Features in Total, 6 Categorical, 4 Numerical

---

---

For our Model with less Quality/Condition Features than SF Features, will contain:<br>
Quality/Condition: `overall_qual`, `overall_cond`,`heating_qc1`,`bsmt_qual`<br>
Squarefeet: `wood_deck_sf`,`lot_area`,`lot_frontage`,`mas_vnr_area`,`garage_area`,`bsmtfin_sf_2` <br>

These features were selected by examining the EDA as well as trying to reduce the multicollinearity we had in the original model. A slight different for one of the numerical features in this model is `bsmtfin_sf_2` whose VIF was much greater than 5, but not infinite. In an effort to make balanced models Numerical/Categorical wise, I needed to dip into higher VIF scoring SQFT features. Once again a hole that the initial hypothesis put us in. As to slimming down the categorical features, I chose to drop both `fireplace_qu` and `heating_qc` based on their distributions<br>

10 Features in Total, 4 Categorical, 6 Numerical

---

Experiment:
Now that we have both our models with opposite amounts of Numerical and Categorical features, we need to get our mean RMSE scores to test our Null and Experimental Hypothesis.
- Each Model will be run 3 different times using 3 different random states (1,42,777).
- Each Model will attempt to run a Multi Linear Regression Model, a RidgeCV Model, and an ElasticNetCv Model
- After each model run, scores will be collected across model types, have their average taken, and test our significant factor

---

In [20]:
df_train_hyp.columns

Index(['id', 'lot_frontage', 'lot_area', 'overall_qual', 'overall_cond',
       'mas_vnr_area', 'bsmtfin_sf_1', 'bsmtfin_sf_2', 'bsmt_unf_sf',
       'total_bsmt_sf', '1st_flr_sf', '2nd_flr_sf', 'low_qual_fin_sf',
       'gr_liv_area', 'garage_area', 'wood_deck_sf', 'open_porch_sf',
       'pool_area', 'saleprice', 'exter_qual', 'exter_cond', 'bsmt_qual',
       'bsmt_cond', 'heating_qc', 'kitchen_qual', 'fireplace_qu',
       'garage_qual', 'garage_cond'],
      dtype='object')

# Model with More Qual/Cond Features

### Preprocessing

In [25]:
hypo_train = df_train_hyp.drop(columns=['bsmtfin_sf_1', 'bsmtfin_sf_2', 'bsmt_unf_sf','total_bsmt_sf',
                                        '1st_flr_sf', '2nd_flr_sf', 'low_qual_fin_sf','gr_liv_area',
                                        'garage_area','open_porch_sf','pool_area','exter_qual',
                                       'exter_cond','bsmt_cond','garage_qual','garage_cond'])

hypo_test = df_test_hyp.drop(columns=['bsmtfin_sf_1', 'bsmtfin_sf_2', 'bsmt_unf_sf','total_bsmt_sf',
                                        '1st_flr_sf', '2nd_flr_sf', 'low_qual_fin_sf','gr_liv_area',
                                        'garage_area','open_porch_sf','pool_area','exter_qual',
                                       'exter_cond','bsmt_cond','garage_qual','garage_cond'])

In [22]:
hypo_train.shape, hypo_test.shape

((2051, 12), (878, 11))

In [27]:
# split dfs into their numerical and object forms for OHE
hypo_train_num = hypo_train.select_dtypes('number')
hypo_train_obj = hypo_train.select_dtypes('object')
hypo_test_num = hypo_test.select_dtypes('number')
hypo_test_obj = hypo_test.select_dtypes('object')

In [30]:
# Fit Transform Train Set
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    hypo_train_obj = ohe.fit_transform(hypo_train_obj)

In [31]:
# Transform Test Set
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    hypo_test_obj = ohe.transform(hypo_test_obj)

In [32]:
# Add index back to object for merge using saved indexes
hypo_train_obj['id'] = train_id
hypo_test_obj['id'] = test_id

In [33]:
# Merge respective dfs back together for 
hypo_ohe_train = hypo_train_num.merge(hypo_train_obj, how='left', on='id')
hypo_ohe_test = hypo_test_num.merge(hypo_test_obj, how='left', on='id')

In [35]:
# Check Shapes after OHE
hypo_ohe_train.shape,hypo_ohe_test.shape

((2051, 24), (878, 23))

In [43]:
# set our X and Y
X = hypo_ohe_train.drop(columns=['saleprice'])
y = hypo_ohe_train['saleprice']

In [132]:
# Test Train Split, with different Random States to get mean RMSE for hypothesis
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=777)

In [133]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1538, 23), (513, 23), (1538,), (513,))

### Feature Engineering

#### Poly - combine variables on an interaction basis (a*b not a**b)

In [134]:
X_train_poly = poly.fit_transform(X_train.drop(columns=['id']))

In [135]:
X_test_poly = poly.transform(X_test.drop(columns=['id']))

In [136]:
X_train_poly.shape, X_test_poly.shape

((1538, 253), (513, 253))

#### StandardScaler - Adjust all numerical features to the same scale

In [137]:
X_train_ss = ss.fit_transform(X_train_poly)

In [138]:
X_test_ss = ss.transform(X_test_poly)

## Modeling

### Base Model

In [139]:
base_1 = [y_train.mean()]*len(y_test)
r2_score(y_test, base_1)

-0.00018676221445179664

Unbelievably our Base Model for our better fit model is even worse than it was when we used all features including features that had 'inf' multicollinearity.

### Multi Linear Regression

In [140]:
mlr = LinearRegression()

In [141]:
mlr.fit(X_train_ss, y_train)

LinearRegression()

Calculate R2 Scores

In [142]:
mlr.score(X_train_ss, y_train)

0.8663915830139093

In [143]:
mlr.score(X_test_ss, y_test)

-3.455706694072568e+22

Calculate RMSE

In [144]:
mean_squared_error(y_train, mlr.predict(X_train_ss), squared = False)

28889.251970539026

In [145]:
mean_squared_error(y_test, mlr.predict(X_test_ss), squared = False)

1.484248680623979e+16

#### Observations:
- #### our better fit model with more qual/cond performed slightly better than our combined model, only slightly in that our test RMSE is positive and not negative
- #### our model continues to be overfit to our training data with less features and a much higher variance

### Ridge CV

In [146]:
alphas = np.logspace(-10, 5, 400)

In [147]:
rg_cv = RidgeCV(alphas=alphas)

In [148]:
rg_cv.fit(X_train_ss, y_train);

In [149]:
# Best Alpha
rg_cv.alpha_

53.619567705688056

In [150]:
%%time
print(f"""
training r2: {rg_cv.score(X_train_ss, y_train)}
validation r2: {rg_cv.score(X_test_ss, y_test)}
cross val score: {cross_val_score(RidgeCV(alphas = rg_cv.alpha_), X_train_ss, y_train).mean()}

training RMSE: {mean_squared_error(y_train, rg_cv.predict(X_train_ss), squared=False)}
validation RMSE: {mean_squared_error(y_test, rg_cv.predict(X_test_ss), squared=False)}
""")


training r2: 0.8549186916703796
validation r2: 0.7865731059575924
cross val score: 0.7919776689512423

training RMSE: 30104.063352292193
validation RMSE: 36886.10570831829

CPU times: user 1.78 s, sys: 439 ms, total: 2.22 s
Wall time: 247 ms


Scores for Random_State = 1
- training r2: 0.8556630430614536
- validation r2: 0.8080414008572958
- cross val score: 0.7741491661321669
- training RMSE: 29673.485338501614
- validation RMSE: 36165.97803392914

Scores for Random State = 42
- training r2: 0.8494395076424457
- validation r2: 0.825969205544636
- cross val score: 0.7912082145042719
- training RMSE: 30858.10880892831
- validation RMSE: 32688.734784197306

Scores for Random State = 777
- training r2: 0.8549186916703796
- validation r2: 0.7865731059575924
- cross val score: 0.7919776689512423
- training RMSE: 30104.063352292193
- validation RMSE: 36886.10570831829

### ElasticNetCv

Train ENCV

In [151]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    enet_cv.fit(X_train_ss, y_train);

Find Best Alpha for ENCV

In [152]:
enet_cv.alpha_

0.04659525668664668

Calculate Scores for ENCV

In [153]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    print(f"""
        training r2: {enet_cv.score(X_train_ss, y_train)}
        validation r2: {enet_cv.score(X_test_ss, y_test)}
        cross val score: {cross_val_score(ElasticNetCV(alphas = [enet_cv.alpha_], n_jobs=-1), X_train_ss, y_train).mean()}

        training RMSE: {mean_squared_error(y_train, enet_cv.predict(X_train_ss), squared=False)}
        validation RMSE: {mean_squared_error(y_test, enet_cv.predict(X_test_ss), squared=False)}
        """)

/Users/christopherjoyce/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 447053992711.9963, tolerance: 628811481.9287658
  model = cd_fast.enet_coordinate_descent_gram(
/Users/christopherjoyce/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 423759656836.5848, tolerance: 635579495.8964632
  model = cd_fast.enet_coordinate_descent_gram(
/Users/christopherjoyce/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 453080231368.94666, tolerance: 630427561.1434635
  model = cd_fast.enet_coordinate_descent_gram(
/Users/christopherjoyce/


        training r2: 0.8535615841399949
        validation r2: 0.7863989676759269
        cross val score: 0.7893952765027602

        training RMSE: 30244.534087147065
        validation RMSE: 36901.15061026141
        


#### Observations:
- #### Initial Observation, with this more feature refined model we were able to actually complete the ElasticNetCV easily

ElasticNetCV
Scores for Random_State = 1
- training r2: 0.8542594092811929
- validation r2: 0.8084967062962131
- cross val score: 0.7722727880825254
- training RMSE: 29817.4191462451
- validation RMSE: 36123.06163838709<br>

Scores for Random State = 42
- training r2: 0.8501690762582578
- validation r2: 0.8259373141311357
- cross val score: 0.7909998105104303
- training RMSE: 30783.253690865084
- validation RMSE: 32691.729777715453

Scores for Random State = 777
- training r2: 0.8535615841399949
- validation r2: 0.7863989676759269
- cross val score: 0.7893952765027602
- training RMSE: 30244.534087147065
- validation RMSE: 36901.15061026141

### Model Final Observations and Analysis

- #### Overall my model with More Qual/Cond Features remained to be overfit to the training data, just as the initial model with all features performed. 
- #### The models r2 scores stay in about the same range of each other across the RidgeCV and ElasticNetCV models, but as above with a trending overfit to the training set.
- #### Across the different Random State changes, the Ridge CV and ElasticNetCV models performed similarly, keep a a range of 2k - 7k difference between training and validation RMSE
- #### To Improve this model in a later project, we should look to use Ordinal Encoder to help delineate the value of the various ordinal series in the Qual/Cond Features.
- #### As of this moment I think the null hypothesis will prevail, unless the Next Model with Less Qual/Cond Features performs even worse than this model


# Model with Less Qual/Cond Features

### Preprocessing

In [154]:
hypo1_train = df_train_hyp.drop(columns=['bsmtfin_sf_1','bsmt_unf_sf','total_bsmt_sf','1st_flr_sf',
                                        '2nd_flr_sf', 'low_qual_fin_sf','gr_liv_area','open_porch_sf',
                                        'pool_area','exter_qual','exter_cond','bsmt_cond',
                                        'garage_qual','garage_cond','fireplace_qu','kitchen_qual'])

hypo1_test = df_test_hyp.drop(columns=['bsmtfin_sf_1','bsmt_unf_sf','total_bsmt_sf','1st_flr_sf',
                                        '2nd_flr_sf', 'low_qual_fin_sf','gr_liv_area','open_porch_sf',
                                        'pool_area','exter_qual','exter_cond','bsmt_cond',
                                        'garage_qual','garage_cond','fireplace_qu','kitchen_qual'])

In [155]:
hypo_train.shape, hypo_test.shape

((2051, 12), (878, 11))

In [156]:
# split dfs into their numerical and object forms for OHE
hypo1_train_num = hypo1_train.select_dtypes('number')
hypo1_train_obj = hypo1_train.select_dtypes('object')
hypo1_test_num = hypo1_test.select_dtypes('number')
hypo1_test_obj = hypo1_test.select_dtypes('object')

In [157]:
# Fit Transform Train Set
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    hypo1_train_obj = ohe.fit_transform(hypo1_train_obj)

In [158]:
# Transform Test Set
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    hypo1_test_obj = ohe.transform(hypo1_test_obj)

In [159]:
# Add index back to object for merge using saved indexes
hypo1_train_obj['id'] = train_id
hypo1_test_obj['id'] = test_id

In [160]:
# Merge respective dfs back together for 
hypo1_ohe_train = hypo1_train_num.merge(hypo1_train_obj, how='left', on='id')
hypo1_ohe_test = hypo1_test_num.merge(hypo1_test_obj, how='left', on='id')

In [161]:
# Check Shapes after OHE
hypo1_ohe_train.shape,hypo1_ohe_test.shape

((2051, 18), (878, 17))

In [249]:
# set our X and Y
X1 = hypo_ohe_train.drop(columns=['saleprice'])
y1 = hypo_ohe_train['saleprice']

In [329]:
# Test Train Split, with different Random States to get mean RMSE for hypothesis
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.25, random_state=777)

### Feature Engineering

#### Poly - combine variables on an interaction basis (a*b not a**b)

In [330]:
X1_train_poly = poly.fit_transform(X1_train.drop(columns=['id']))

In [331]:
X1_test_poly = poly.transform(X1_test.drop(columns=['id']))

#### StandardScaler - Adjust all numerical features to the same scale

In [332]:
X1_train_ss = ss.fit_transform(X1_train_poly)

In [333]:
X1_test_ss = ss.transform(X1_test_poly)

In [334]:
X1_train_ss.shape,X1_test_ss.shape

((1538, 253), (513, 253))

### Modeling

### Base Model

In [335]:
base_1 = [y1_train.mean()]*len(y1_test)
r2_score(y1_test, base_1)

-0.00018676221445179664

In [336]:
mlr1 = LinearRegression()

In [337]:
mlr1.fit(X1_train_ss, y_train1);

In [338]:
mlr1.score(X1_train_ss, y1_train)

-0.05649165250497434

In [339]:
mlr1.score(X1_test_ss, y1_test)

-4.286512045847645e+24

In [340]:
mean_squared_error(y1_train, mlr.predict(X1_train_ss), squared = False)

28889.251970539026

In [341]:
mean_squared_error(y1_test, mlr.predict(X1_test_ss), squared = False)

1.484248680623979e+16

Initial Observation(random_state=1): I think from this performance alone the model with more squarefeet fields is going to outperform the model with more qual/cond fields. The r2 scores are still reminiscent of being overfit to the training data, and while the RMSE is still extremely high, both of the values from the MLR model are the same length digits and both positive.

### Ridge CV

In [342]:
alphas = np.logspace(-10, 5, 400)

In [343]:
rg_cv1 = RidgeCV(alphas=alphas)

In [344]:
rg_cv1.fit(X1_train_ss, y1_train);

In [345]:
# Best Alpha
rg_cv1.alpha_

53.619567705688056

In [346]:
%%time
print(f"""
training r2: {rg_cv1.score(X1_train_ss, y1_train)}
validation r2: {rg_cv1.score(X1_test_ss, y1_test)}
cross val score: {cross_val_score(RidgeCV(alphas = rg_cv1.alpha_), X1_train_ss, y1_train).mean()}
training RMSE: {mean_squared_error(y1_train, rg_cv1.predict(X1_train_ss), squared=False)}
validation RMSE: {mean_squared_error(y1_test, rg_cv1.predict(X1_test_ss), squared=False)}
""")


training r2: 0.8549186916703796
validation r2: 0.7865731059575924
cross val score: 0.7919776689512423
training RMSE: 30104.063352292193
validation RMSE: 36886.10570831829

CPU times: user 1.79 s, sys: 366 ms, total: 2.15 s
Wall time: 240 ms


Scores for Random_State = 1
- training r2: 0.8556630430614536
- validation r2: 0.8080414008572958
- cross val score: 0.7741491661321669
- training RMSE: 29673.485338501614
- validation RMSE: 36165.97803392914

Scores for Random State = 42
- training r2: 0.8494395076424457
- validation r2: 0.825969205544636
- cross val score: 0.7912082145042719
- training RMSE: 30858.10880892831
- validation RMSE: 32688.734784197306

Scores for Random State = 777
- training r2: 0.8549186916703796
- validation r2: 0.7865731059575924
- cross val score: 0.7919776689512423
- training RMSE: 30104.063352292193
- validation RMSE: 36886.10570831829

#### Observations:
- #### Both of the below observations are incorrect, after scanning my code for errors, I found that i had not changed some of the test variables correctly. After fixing and rerunning the code, both models are performing exactly the same.
    - #### My initial observation after seeing the mlr model perform better on more square feet variables was incorrect, the model with more Qual/Cond Features is definitely outperforming the model with more SQFT Variables
    - #### What I find interesting is the r2 score ranges are pretty similar, but the RMSE ranges are astronomically different

### ElasticNetCV

In [347]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    enet_cv1.fit(X1_train_ss, y1_train);

In [348]:
enet_cv1.alpha_

0.04659525668664668

In [349]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    print(f"""
        training r2: {enet_cv1.score(X1_train_ss, y1_train)}
        validation r2: {enet_cv1.score(X1_test_ss, y1_test)}
        cross val score: {cross_val_score(ElasticNetCV(alphas = [enet_cv1.alpha_], n_jobs=-1), X1_train_ss, y1_train).mean()}
        training RMSE: {mean_squared_error(y1_train, enet_cv1.predict(X1_train_ss), squared=False)}
        validation RMSE: {mean_squared_error(y1_test, enet_cv1.predict(X1_test_ss), squared=False)}
        """)

/Users/christopherjoyce/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 446915201768.9233, tolerance: 617539356.6888041
  model = cd_fast.enet_coordinate_descent_gram(
/Users/christopherjoyce/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 453080231368.94666, tolerance: 630427561.1434635
  model = cd_fast.enet_coordinate_descent_gram(
/Users/christopherjoyce/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 447053992711.9963, tolerance: 628811481.9287658
  model = cd_fast.enet_coordinate_descent_gram(
/Users/christopherjoyce/


        training r2: 0.8535615841399949
        validation r2: 0.7863989676759269
        cross val score: 0.7893952765027602
        training RMSE: 30244.534087147065
        validation RMSE: 36901.15061026141
        


ElasticNetCV
Scores for Random_State = 1
- training r2: 0.8542594092811929
- validation r2: 0.8084967062962131
- cross val score: 0.7722727880825254
- training RMSE: 29817.4191462451
- validation RMSE: 36123.06163838709

Scores for Random State = 42
- training r2: 0.8501690762582578
- validation r2: 0.8259373141311357
- cross val score: 0.7909998105104303
- training RMSE: 30783.253690865084
- validation RMSE: 32691.729777715453

Scores for Random State = 777
- training r2: 0.8535615841399949
- validation r2: 0.7863989676759269
- cross val score: 0.7893952765027602
- training RMSE: 30244.534087147065
- validation RMSE: 36901.15061026141


## Modeling Insights and Defense:
- #### What I learned from this hypothesis and exercise is the importance of taking care with how you choose to manipulate the data. In this instance I chose to impute Median for Numerical and Mode for Categorical features. Even further I only chose to One Hot Encode rather than trying to continue to figure out Ordinal Encoder which I believe would have benefited the Qual/Condition Features greatly.
- #### Both versions of the model performed, at some points (look at the scores..) exactly the same. I can't with much confidence support either model as they performed similarly.
- #### Something I had a hard time combatting in this model and the models I built for the Kaggle competition was the bias of a given data set. Finding the right balance between bias and variance, more or less features seemed to be a delicate balance because my scores more or less stayed consistent no matter the action taken.
- #### A mistake I caught which explains why the ElasticNetCV was performing similarly to the ridge was the l1_ratio field. I kept the values closer to Zero which balances the ENCV model to act more like a Ridge than a Lasso model. Next time I pick up this project I will look to better balance the l1_ratio to include the LassoCV side of the model.
- #### The final lesson is to not be so strict on what you consider certain categories of features re what constitutes a quality/condition feature and what does not. After drilling further into the data dictionary supplied by Kaggle, I noticed that Basement Qual isnt actually what I thought it meant; it is related to the height of the basement ceiling and not actually the quality of the basement itself.

# Hypothesis Testing and Conclusion

Model with more Qual/Cond Features RidgeCV & ElasticNetCV mean RMSE across the three trials:

In [244]:
train_ridge_cv_rmse_mean = (29673.49+30858.11+30104.06)/3

In [245]:
test_ridge_cv_rmse_mean = (36165.98+32688.73+36886.11)/3

In [350]:
train_encv_rmse_mean = (29817.41+30783.25+30244.53)/3

In [351]:
test_encv_rmse_mean = (36123+32691.73+36901.15)/3

Model with less Qual/Cond Features RidgeCV & ElasticNetCV mean RMSE across the three trials:

In [352]:
train_ridge_cv_rmse_mean1 = (29673.5+30858.11+30104.06)/3

In [353]:
test_ridge_cv_rmse_mean1 = (36165.97+32688.73+36886.11)/3

In [354]:
train_encv_rmse_mean1 = (29817.42+30783.25+30244.53)/3

In [355]:
test_encv_rmse_mean1 = (36123.06+32691.73+36901.15)/3

Does the Null Hypothesis hold? 

In [356]:
train_ridge_cv_rmse_mean-train_ridge_cv_rmse_mean1

-0.003333333330374444

In [357]:
test_ridge_cv_rmse_mean-test_ridge_cv_rmse_mean1

0.0033333333340124227

In [358]:
train_encv_rmse_mean-train_encv_rmse_mean1

-0.0033333333340124227

In [360]:
test_encv_rmse_mean-test_encv_rmse_mean1

-0.01999999999679858

# Hypothesis Conclusion Statement:<br>
### Because none of the differences taken by mean RMSE scores were larger than our level of significance = 0.05, there is insufficient evidence to reject our null hypothesis. We do not have enough evidence to conclude that Quality and Condition Features are better predictors of sale price than Squarefeet based features.

# Conclusions and Recommendations

## Conclusions:
- ### From the current iteration of the model and data, there is not ample enough evidence to suggest that Quality/Condition nor SquareFt Feature based fields are the best predictors of price.
- ### Quality and Condition Features range further than just Features that reference quality or condition in their name.
- ### Squarefeet based features are also not entirely reliable on predicting potential sale price points either. As seen in EDA and further the performance of the models, Squarefeet Features have a very high multicollinearity with other Squarefeet Features which ultimately creates more noise than it does highlight the signal.
- ### Despite postive Linear relationships between individual Quality,Condition,Squarefeet features and Sale Price, more features need to be included in order to accurately predict sale prices that better tie the entire picture together, potentially such as numbers of category rooms (baths, bedrooms, etc.) as well as adding features that detail home/bldg style.

## Recommendations:
- ### Fund another round of analysis on the data to take lessons learned and reapply to better understand the data and create a model that is less restrictive of features it includes.
- ### When creating future models that predict price, ensure that the parameters set in cross validation models don't test models in a similar fasion.
- ### Continue to focus on features that represent non-depcreciative value such as Squarefeet as they tend to have strong positive correlations with price.
- ### When considering Quality/Condition Features, focus on instances of Heating quality and basement height as they are more evenly distributed than their peer features such as Garage Quality, Kitchen Quality, and Fireplace quality.